In [2]:
import numpy as np
import pandas as pd
from io import BytesIO

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from data_preprocessing import get_input, get_input_share_tokenizer

from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import regularizers

In [3]:
def create_model(vocabulary_length, max_length_sequence, emb_dim, filters, kernel_size, dense_units, l2_kernel):
    
    X_input = Input(shape = (max_length_sequence, ))
    embedding_layer = Embedding(input_dim = vocabulary_length, output_dim = emb_dim,
                                trainable = True)(X_input)
    
    X = Conv1D(filters = filters, kernel_size = kernel_size, activation = 'relu',
              kernel_regularizer = regularizers.l2(l2_kernel))(embedding_layer)
    X = MaxPooling1D(pool_size = 2)(X)
    X = Flatten()(X)
    X = Dense(units = dense_units, activation = 'relu')(X)
    X = Dense(units = 1, activation = 'sigmoid')(X)
                          
    model = Model(inputs = X_input, outputs = X)
                          
    return(model)

In [4]:
model = create_model(10000, 500, 100, filters = 32, kernel_size = 8, dense_units = 10, l2_kernel = 0)
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 493, 32)           25632     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 246, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7872)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                78730     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11 

In [11]:
def grid_search(X, dataset, epochs, models):
    
    Y = dataset.label.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)

    mean_dev_accuracy = np.zeros(np.size(models))
    
    i = 0
    for model in models:
    
        iterations = 5
        dev_accuracy_shuffle_split = np.zeros(iterations)
        shuffle = ShuffleSplit(n_splits = iterations, test_size = 0.2)
        
        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model
        w = BytesIO()
        model.save_weights(w) # Save initial weights
        
        j = 0
        for train, dev in shuffle.split(X_train, y_train):
            Xtrain = X_train[train]
            Ytrain = y_train[train]
            Xdev = X_train[dev]
            Ydev = y_train[dev]

            model.fit(Xtrain, Ytrain, epochs = epochs, batch_size = 32, shuffle = True) # Fit model

            loss, accuracy_val = model.evaluate(Xdev, Ydev) # Validate model
            dev_accuracy_shuffle_split[j] = accuracy_val

            model.load_weights(w) # Restore initial weights
            
            j += 1
              
        mean_dev_accuracy[i] = round(np.mean(dev_accuracy_shuffle_split), 3)
        std = round(np.std(dev_accuracy_shuffle_split), 3)
        print('Model ' + str(i) +' --> dev_acc: ' + str(mean_dev_accuracy[i]) + ' +- ' + str(std))
        i += 1
    
    # test
    best_model_index = np.argmax(mean_dev_accuracy)
    best_model = models[best_model_index]
    best_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile best model
    best_model.fit(X_train, y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit best model
    y_pred = best_model.predict(X_test) # Test best model
    y_pred = y_pred > 0.5 # Sigmoid activation function
    accuracy_test = accuracy_score(y_test, y_pred)
    
    return best_model_index, round(accuracy_test, 3)

In [5]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
...,...,...
2566,"Recuperamos la historia de Aleixandra, la jove...",0
2567,"Reproches, tensión y sinceridad: la comida en ...",0
2568,"RT @ElMundoOpinion: ""PSOE, PP, Ciudadanos y Vo...",0
2569,Rusia cita al embajador español por unas decla...,0


#### Models

In [6]:
vocabulary_length = 10000
max_length_sequence = 500
emb_dim = 100
language = 'spanish'
epochs = 7

In [7]:
X, df = get_input(dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X.shape)

(2571, 500)


In [8]:
models = []
filters = [8, 16, 32]
kernel_size = [5, 10]
dense_units = [4, 8, 12]

for i in range(len(filters)):
    for j in range(len(kernel_size)):
        for k in range(len(dense_units)):
            model = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = filters[i], kernel_size = kernel_size[j], dense_units = dense_units[k], l2_kernel = 0)
            models.append(model)

In [9]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6888 - accuracy: 0.5189
Epoch 2/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6345 - accuracy: 0.7603
Epoch 3/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.4427 - accuracy: 0.9075
Epoch 4/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.1507 - accuracy: 0.9751
Epoch 5/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.0400 - accuracy: 0.9921
Epoch 6/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.0164 - accuracy: 0.9976
Epoch 7/7
412/412 [==============================] - 0s 536us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6906 - accuracy: 0.5140
Epoch 2/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6523 - accuracy: 0.5943
Epoch 3/7
1644/1644 [=========

1644/1644 [==============================] - 5s 3ms/step - loss: 0.0465 - accuracy: 0.9933
Epoch 5/7
1644/1644 [==============================] - 4s 3ms/step - loss: 0.0153 - accuracy: 0.9970
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0071 - accuracy: 0.9988
Epoch 7/7
412/412 [==============================] - 0s 649us/step
Epoch 1/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6885 - accuracy: 0.5353
Epoch 2/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6090 - accuracy: 0.7597
Epoch 3/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.4073 - accuracy: 0.8929
Epoch 4/7
1644/1644 [==============================] - 6s 4ms/step - loss: 0.1395 - accuracy: 0.9732
Epoch 5/7
1644/1644 [==============================] - 6s 3ms/step - loss: 0.0302 - accuracy: 0.9939
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0107 - accuracy: 0.9988
Epoch 7/7
412/412 [===============

1644/1644 [==============================] - 4s 2ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 6/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 7/7
412/412 [==============================] - 0s 354us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step - loss: 0.6863 - accuracy: 0.5255
Epoch 2/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.6055 - accuracy: 0.6995
Epoch 3/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.3799 - accuracy: 0.9136
Epoch 4/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0845 - accuracy: 0.9903
Epoch 5/7
1644/1644 [==============================] - 4s 2ms/step - loss: 0.0107 - accuracy: 0.9982
Epoch 6/7
1644/1644 [==============================] - 4s 3ms/step - loss: 0.0041 - accuracy: 0.9994
Epoch 7/7
412/412 [==============================] - 0s 355us/step
Epoch 1/7
1644/1644 [==============================] - 3s 2ms/step -

1644/1644 [==============================] - 5s 3ms/step - loss: 0.0104 - accuracy: 0.9982
Epoch 7/7
412/412 [==============================] - 0s 728us/step
Epoch 1/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6838 - accuracy: 0.5298
Epoch 2/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.6269 - accuracy: 0.6527
Epoch 3/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.4197 - accuracy: 0.8771
Epoch 4/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.1073 - accuracy: 0.9757
Epoch 5/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0143 - accuracy: 0.9988
Epoch 6/7
1644/1644 [==============================] - 5s 3ms/step - loss: 0.0054 - accuracy: 0.9988
Epoch 7/7
412/412 [==============================] - 0s 656us/step
Epoch 1/7
1644/1644 [==============================] - 7s 4ms/step - loss: 0.6937 - accuracy: 0.5073
Epoch 2/7
1644/1644 [==============================] - 6s 4ms/step -

412/412 [==============================] - 1s 2ms/step
Epoch 1/7
1644/1644 [==============================] - 10s 6ms/step - loss: 0.6896 - accuracy: 0.5030
Epoch 2/7
1644/1644 [==============================] - 11s 6ms/step - loss: 0.6640 - accuracy: 0.5633
Epoch 3/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.5546 - accuracy: 0.7336
Epoch 4/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.3279 - accuracy: 0.8753
Epoch 5/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.0781 - accuracy: 0.9757
Epoch 6/7
1644/1644 [==============================] - 9s 6ms/step - loss: 0.0223 - accuracy: 0.9945
Epoch 7/7
412/412 [==============================] - 1s 1ms/step
Model 16 --> dev_acc: 0.714 +- 0.021
Epoch 1/7
1644/1644 [==============================] - 10s 6ms/step - loss: 0.6769 - accuracy: 0.5377
Epoch 2/7
1644/1644 [==============================] - 8s 5ms/step - loss: 0.5836 - accuracy: 0.7068
Epoch 3/7
1644/1644 [===========

In [8]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 4, l2_kernel = 0.01)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 4, l2_kernel = 0.1)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 4, l2_kernel = 1)

models.append(model_0)
models.append(model_1)
models.append(model_2)

In [9]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

C:\Users\Kevin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
1644/1644 [==============================] - 17s 10ms/step - loss: 0.7981 - accuracy: 0.5079
Epoch 2/7
1644/1644 [==============================] - 14s 9ms/step - loss: 0.6724 - accuracy: 0.5529
Epoch 3/7
1644/1644 [==============================] - 13s 8ms/step - loss: 0.6283 - accuracy: 0.5870
Epoch 4/7
1644/1644 [==============================] - 14s 9ms/step - loss: 0.5354 - accuracy: 0.6551
Epoch 5/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.4038 - accuracy: 0.6983
Epoch 6/7
1644/1644 [==============================] - 16s 10ms/step - loss: 0.2496 - accuracy: 0.9556
Epoch 7/7
412/412 [==============================] - 1s 2ms/step
Epoch 1/7
1644/1644 [==============================] - 17s 10ms/step - loss: 0.8464 - accuracy: 0.5335
Epoch 2/7
1644/1644 [==============================] - 17s 10ms/step - loss: 0.7049 - accuracy: 0.5864
Epoch 3/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.6008 - accuracy: 0.6904
Epoch 4/7
164

C:\Users\Kevin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
1644/1644 [==============================] - 16s 10ms/step - loss: 1.7856 - accuracy: 0.5164
Epoch 2/7
1644/1644 [==============================] - 14s 8ms/step - loss: 0.7352 - accuracy: 0.5414
Epoch 3/7
1644/1644 [==============================] - 14s 9ms/step - loss: 0.6779 - accuracy: 0.5426
Epoch 4/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.6593 - accuracy: 0.5955
Epoch 5/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.6231 - accuracy: 0.7105
Epoch 6/7
1644/1644 [==============================] - 14s 8ms/step - loss: 0.5363 - accuracy: 0.8005
Epoch 7/7
412/412 [==============================] - 1s 2ms/step
Epoch 1/7
1644/1644 [==============================] - 14s 9ms/step - loss: 1.3956 - accuracy: 0.5012
Epoch 2/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.6948 - accuracy: 0.5036
Epoch 3/7
1644/1644 [==============================] - 13s 8ms/step - loss: 0.6931 - accuracy: 0.5036
Epoch 4/7
1644/1

C:\Users\Kevin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
1644/1644 [==============================] - 18s 11ms/step - loss: 11.0326 - accuracy: 0.5182
Epoch 2/7
1644/1644 [==============================] - 21s 13ms/step - loss: 1.1072 - accuracy: 0.5055
Epoch 3/7
1644/1644 [==============================] - 17s 10ms/step - loss: 0.6977 - accuracy: 0.5493
Epoch 4/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.6957 - accuracy: 0.5426
Epoch 5/7
1644/1644 [==============================] - 16s 9ms/step - loss: 0.6913 - accuracy: 0.5401
Epoch 6/7
1644/1644 [==============================] - 16s 10ms/step - loss: 0.6819 - accuracy: 0.5706
Epoch 7/7
412/412 [==============================] - 1s 2ms/step
Epoch 1/7
1644/1644 [==============================] - 13s 8ms/step - loss: 7.0589 - accuracy: 0.5195
Epoch 2/7
1644/1644 [==============================] - 15s 9ms/step - loss: 0.7000 - accuracy: 0.5347
Epoch 3/7
1644/1644 [==============================] - 14s 8ms/step - loss: 0.6898 - accuracy: 0.5669
Epoch 4/7
16

C:\Users\Kevin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
2056/2056 [==============================] - 19s 9ms/step - loss: 0.7835 - accuracy: 0.5136
Epoch 2/7
2056/2056 [==============================] - 18s 9ms/step - loss: 0.6697 - accuracy: 0.5661
Epoch 3/7
2056/2056 [==============================] - 19s 9ms/step - loss: 0.5391 - accuracy: 0.7232
Epoch 4/7
2056/2056 [==============================] - 21s 10ms/step - loss: 0.2723 - accuracy: 0.9217
Epoch 5/7
2056/2056 [==============================] - 19s 9ms/step - loss: 0.1163 - accuracy: 0.9805
Epoch 6/7
2056/2056 [==============================] - 18s 9ms/step - loss: 0.0663 - accuracy: 0.9927
Epoch 7/7
2056/2056 [==============================] - 19s 9ms/step - loss: 0.0486 - accuracy: 0.9956
best_model_index: 0 test acc: 0.685


#### Test CNN Model with English Dataset

In [12]:
dataset = pd.read_csv('../data/Merged/english_dataset.csv')
dataset

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
5,The number of cases of cops brutalizing and ki...,1
6,Donald Trump spent a good portion of his day a...,1
7,In the wake of yet another court decision that...,1
8,Many people have raised the alarm regarding th...,1
9,Just when you might have thought we d get a br...,1


In [13]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 100
language = 'english'
epochs = 7

In [14]:
X, df = get_input(dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X.shape)

(51233, 1500)


In [15]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 4, l2_kernel = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 8, l2_kernel = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0)
model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0.01)


models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)

In [16]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/7
32788/32788 [==============================] - 687s 21ms/step - loss: 0.5680 - accuracy: 0.6193
Epoch 2/7
32788/32788 [==============================] - 493s 15ms/step - loss: 0.0570 - accuracy: 0.9807
Epoch 3/7
32788/32788 [==============================] - 402s 12ms/step - loss: 0.0145 - accuracy: 0.9959
Epoch 4/7
32788/32788 [==============================] - 333s 10ms/step - loss: 0.0048 - accuracy: 0.9987
Epoch 5/7
32788/32788 [==============================] - 324s 10ms/step - loss: 0.0026 - accuracy: 0.9994
Epoch 6/7
32788/32788 [==============================] - 314s 10ms/step - loss: 0.0035 - accuracy: 0.9990
Epoch 7/7
8198/8198 [==============================] - 16s 2ms/step
Epoch 1/7
32788/32788 [==============================] - 335s 10ms/step - loss: 0.1552 - accuracy: 0.9339
Epoch 2/7
32788/32788 [==============================] - 319s 10ms/step - loss: 0.0200 - accuracy: 0.9941
Epoch 3/7
32788/32788 [==============================] - 324s 10ms/step - loss: 0.00

40986/40986 [==============================] - 326s 8ms/step - loss: 0.0062 - accuracy: 0.9983
Epoch 4/7
40986/40986 [==============================] - 326s 8ms/step - loss: 0.0052 - accuracy: 0.9984
Epoch 5/7
40986/40986 [==============================] - 330s 8ms/step - loss: 0.0038 - accuracy: 0.9989
Epoch 6/7
40986/40986 [==============================] - 345s 8ms/step - loss: 0.0040 - accuracy: 0.9988
Epoch 7/7
40986/40986 [==============================] - 344s 8ms/step - loss: 0.0039 - accuracy: 0.9989
best_model_index: 2 test acc: 0.982


#### Train Model with English Dataset and Evaluate with Translated Dataset

In [3]:
english_dataset = pd.read_csv('../data/Merged/english_dataset.csv')
english_dataset

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
...,...,...
51228,The State Department told the Republican Natio...,0
51229,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,1
51230,Anti-Trump Protesters Are Tools of the Oligar...,1
51231,"ADDIS ABABA, Ethiopia —President Obama convene...",0


In [4]:
translated_dataset = pd.read_csv('../data/Merged/spanish_t_dataset.csv')
translated_dataset

,text,label
0,"RAE WILL INCLUDE THE WORD ""LADY"" IN THE SPANIS...",1
1,"The word ""haiga"", accepted by the RAE The Roya...",1
2,YORDI ROSADO WILL WRITE AND DESIGN THE NEW SEP...,1
3,UNAM will train teachers to pass the Pisa test...,0
4,Alert: they intend to approve school books wit...,1
...,...,...
2566,"We recover the story of Aleixandra, the 21-yea...",0
2567,"Reproaches, tension and sincerity: the meal in...",0
2568,"RT @ElMundoOpinion: ""PSOE, PP, Ciudadanos and ...",0
2569,Russia quotes the Spanish ambassador for some ...,0


In [5]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 100
language = 'english'
epochs = 7

In [23]:
X_train, df = get_input(english_dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X_train.shape)

(51233, 1500)


In [24]:
X_test, df = get_input(translated_dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X_test.shape)

(2571, 1500)


In [25]:
model = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model

In [26]:
Y_train = english_dataset.label.values
model.fit(X_train, Y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit model

Epoch 1/7
51233/51233 [==============================] - 426s 8ms/step - loss: 0.0834 - accuracy: 0.9674
Epoch 2/7
51233/51233 [==============================] - 405s 8ms/step - loss: 0.0202 - accuracy: 0.9940
Epoch 3/7
51233/51233 [==============================] - 399s 8ms/step - loss: 0.0076 - accuracy: 0.9979
Epoch 4/7
51233/51233 [==============================] - 401s 8ms/step - loss: 0.0042 - accuracy: 0.9986
Epoch 5/7
51233/51233 [==============================] - 401s 8ms/step - loss: 0.0046 - accuracy: 0.9987
Epoch 6/7
51233/51233 [==============================] - 402s 8ms/step - loss: 0.0037 - accuracy: 0.9990
Epoch 7/7
51233/51233 [==============================] - 402s 8ms/step - loss: 0.0021 - accuracy: 0.9996


In [27]:
Y_test = translated_dataset.label.values
loss, acc = model.evaluate(X_test, Y_test)
print(loss, round(acc, 3))

2571/2571 [==============================] - 5s 2ms/step
5.692464059542608 0.519


##### Sharing Tokenizer

In [5]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 100
language = 'english'
epochs = 7

In [6]:
english_dataset = pd.read_csv('../data/Merged/english_dataset.csv')
translated_dataset = pd.read_csv('../data/Merged/spanish_t_dataset.csv')

In [7]:
X_train, X_test, df1, df2 = get_input_share_tokenizer(english_dataset, translated_dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X_train.shape, X_test.shape)

(51233, 1500) (2571, 1500)


In [8]:
model = create_model(vocabulary_length, max_length_sequence, emb_dim, filters = 16, kernel_size = 10, dense_units = 12, l2_kernel = 0)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
Y_train = english_dataset.label.values
model.fit(X_train, Y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit model
Y_test = translated_dataset.label.values
loss, acc = model.evaluate(X_test, Y_test)
print(loss, round(acc, 3))


Epoch 1/7
51233/51233 [==============================] - 431s 8ms/step - loss: 0.0899 - accuracy: 0.9623
Epoch 2/7
51233/51233 [==============================] - 407s 8ms/step - loss: 0.0168 - accuracy: 0.9950
Epoch 3/7
51233/51233 [==============================] - 403s 8ms/step - loss: 0.0114 - accuracy: 0.9969
Epoch 4/7
51233/51233 [==============================] - 403s 8ms/step - loss: 0.0056 - accuracy: 0.9985
Epoch 5/7
51233/51233 [==============================] - 403s 8ms/step - loss: 0.0059 - accuracy: 0.9988
Epoch 6/7
51233/51233 [==============================] - 406s 8ms/step - loss: 0.0035 - accuracy: 0.9990
Epoch 7/7
2571/2571 [==============================] - 4s 1ms/step
4.962719830220332 0.532
